In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pathlib as pl
import flopy

In [ ]:
# import containerized functionality from defaults.py and performance
from defaults import *
from performance import *

In [ ]:
figure_path = pl.Path("figures")
figure_path.mkdir(parents=True, exist_ok=True)

### Parallel settings

1. Set voronoi to `True` to evaluate performance of Voronoi grid base and parallel models. metis cannot be `True` if voronoi is `True`.
2. Set metis to `True` to evaluate performance of structured grid base and parallel models split with Metis. voronoi cannot be `True` if metis is `True`.

In [ ]:
voronoi = False
metis = True

In [ ]:
paths = get_available_workspaces(metis=metis, voronoi=voronoi)
paths

In [ ]:
base_sim = flopy.mf6.MFSimulation.load(
    sim_name="basin",
    sim_ws=paths[0],
    verbosity_level=0,
)
base_gwf = base_sim.get_model()

In [ ]:
total_cells, active_cells = get_simulation_cell_count(base_sim)
total_cells, active_cells

In [ ]:
processors = get_simulation_processors(metis=metis, voronoi=voronoi)
processors

### Evaluate if there are any failed simulations

In [ ]:
processors_remove = []
paths_remove = []
for idx, path in enumerate(paths):
    list_files = get_simulation_listfiles(path)
    if len(list_files) == 0:
        processors_remove.append(processors[idx])
        paths_remove.append(path)
        continue
    for list_file in list_files:
        sim_data = SimulationData(list_file)
        if not sim_data.is_normal_termination():
            if path not in paths_remove:
                processors_remove.append(processors[idx])
                paths_remove.append(path)
            break
if len(processors_remove) > 0:
    print("Processors to remove:", ", ".join(str(v) for v in processors_remove))
    print("Paths to remove:", ", ".join(str(s) for s in paths_remove))

In [ ]:
for proc, pth in zip(processors_remove, paths_remove):
    processors.remove(proc)
    paths.remove(pth)

### Calculate the performance data for the simulations

In [ ]:
# calculate performance data
mean_runtimes = []
mean_formulatetimes = []
mean_solutiontimes = []
max_iterations = []
total_memory_usage = []
max_memory_usage = []
for idx, path in enumerate(paths):
    list_files = get_simulation_listfiles(path)
    mean_runtimes.append(
        np.mean(
            [
                SimulationData(list_file).get_model_runtime()
                for list_file in list_files
            ]
        )
    )
    mean_formulatetimes.append(
        np.mean(
            [
                SimulationData(list_file).get_formulate_time()
                for list_file in list_files
            ]
        )
    )
    mean_solutiontimes.append(
        np.mean(
            [
                SimulationData(list_file).get_solution_time()
                for list_file in list_files
            ]
        )
    )
    max_iterations.append(
        int(
            np.max(
                [
                    SimulationData(list_file).get_total_iterations()
                    for list_file in list_files
                    ]
            )
        )
    )
    total_memory_usage.append(
        np.sum(
            [
                SimulationData(list_file).get_memory_usage()
                for list_file in list_files
            ]
        )
    )
    max_memory_usage.append(
        np.max(
            [
                SimulationData(list_file).get_memory_usage()
                for list_file in list_files
            ]
        )
    )
    

In [ ]:
if len(paths) > 0:
    speedup = (mean_runtimes[0] / np.array(mean_runtimes)).tolist()
    formulate_speedup = (mean_formulatetimes[0] / np.array(mean_formulatetimes)).tolist()
    solution_speedup = (mean_solutiontimes[0] / np.array(mean_solutiontimes)).tolist()
else:
    speedup = []
    formulate_speedup = []
    solution_speedup = []

In [ ]:
# create a json for data
performance = {
    "grid_type": base_gwf.modelgrid.grid_type,
    "metis": metis,
    "voronoi": voronoi,
    "total_cells": total_cells,
    "active_cells": active_cells,
    "processors": processors,
    "mean_runtimes": mean_runtimes,
    "max_iterations": max_iterations,
    "mean_formulatetimes": mean_formulatetimes,
    "mean_solutiontimes": mean_solutiontimes,
    "formulate_speedup": formulate_speedup,
    "solution_speedup": solution_speedup,
    "speedup": speedup,
    "max_memory_usage": max_memory_usage,
    "total_memory_usage": total_memory_usage,
}

### Save the performance data to a json

In [ ]:
performance_json = save_performance_json(performance)